<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/AI_agents/Text_to_SQL_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Libraries

In [1]:
!pip install llama-index llama-index-llms-openai openai sqlalchemy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Generate DB

In [3]:
from sqlalchemy import create_engine,MetaData,Table,Column,String,Integer,select,Boolean,Date,Float

In [4]:
# Create an in-memory SQLite database engine using SQLAlchemy's create_engine function.
# The ':memory:' argument indicates that the database is temporary and stored in RAM.
engine = create_engine("sqlite:///:memory:")

# Initialize a MetaData object, which will hold the schema definitions for the database.
# MetaData is used to track information about tables, columns, constraints, etc.
metadata_obj = MetaData()

In [5]:
# Define the 'employee' table using SQLAlchemy's Table function.
employee_table = Table(
    "employee",  # Table name
    metadata_obj,  # The metadata object that will track the table schema
    Column("employee_id", Integer, primary_key=True),  # Unique ID for each employee, primary key
    Column("first_name", String(50), nullable=False),  # First name of the employee, required field
    Column("last_name", String(50), nullable=False),  # Last name of the employee, required field
    Column("email", String(100), unique=True),  # Email, must be unique for each employee
    Column("phone_number", String(15)),  # Phone number, optional field
    Column("hire_date", Date, nullable=False),  # Date of hiring, required field
    Column("job_title", String(50)),  # Job title of the employee, optional field
    Column("salary", Float),  # Employee's salary, optional field
    Column("is_manager", Boolean, default=False),  # Boolean indicating if the employee is a manager, default is False
)

# Create the 'employee' table in the database associated with the engine.
metadata_obj.create_all(engine)

In [7]:
# SQLDatabase will be used to interact with a SQL database
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

llm = OpenAI(temperature=0.1, model="gpt-4o-mini")

# The 'include_tables' argument ensures that only the "employee" table is considered for querying.
sql_database = SQLDatabase(engine, include_tables=["employee"])

In [8]:
from sqlalchemy import insert
from datetime import date

# Define a list of dictionaries representing rows to insert into the 'employee' table.
# Each dictionary contains the details of an employee such as their ID, name, email, etc.
rows = [
    {
        "employee_id": 1,
        "first_name": "Alice",
        "last_name": "Johnson",
        "email": "alice.johnson@example.com",
        "phone_number": "123-456-7890",
        "hire_date": date(2021, 6, 15),
        "job_title": "Software Developer",
        "salary": 85000.0,
        "is_manager": False,
    },
    {
        "employee_id": 2,
        "first_name": "Bob",
        "last_name": "Smith",
        "email": "bob.smith@example.com",
        "phone_number": "987-654-3210",
        "hire_date": date(2020, 3, 10),
        "job_title": "Project Manager",
        "salary": 120000.0,
        "is_manager": True,
    },
    {
        "employee_id": 3,
        "first_name": "Charlie",
        "last_name": "Brown",
        "email": "charlie.brown@example.com",
        "phone_number": "555-123-4567",
        "hire_date": date(2019, 11, 1),
        "job_title": "Data Scientist",
        "salary": 95000.0,
        "is_manager": False,
    },
    {
        "employee_id": 4,
        "first_name": "Diana",
        "last_name": "Prince",
        "email": "diana.prince@example.com",
        "phone_number": "333-444-5555",
        "hire_date": date(2022, 7, 20),
        "job_title": "HR Specialist",
        "salary": 70000.0,
        "is_manager": False,
    },
    {
        "employee_id": 5,
        "first_name": "Evan",
        "last_name": "Taylor",
        "email": "evan.taylor@example.com",
        "phone_number": "222-333-4444",
        "hire_date": date(2023, 1, 5),
        "job_title": "Marketing Analyst",
        "salary": 65000.0,
        "is_manager": False,
    },
]

# Loop over each row in the 'rows' list to insert data into the employee_table.
for row in rows:
    # Create an insert statement for the employee_table with the data from each row.
    alchemy_query = insert(employee_table).values(**row)

    # Execute the insert statement within a transaction block.
    # 'engine.begin()' ensures that the database connection is committed or rolled back correctly.
    with engine.begin() as connection:
        cursor = connection.execute(alchemy_query)  # Execute the insert statement on the connection

In [9]:
# Sample data read
alchemy_query = select(
    employee_table.c.employee_id,
    employee_table.c.first_name,
    employee_table.c.last_name
).select_from(employee_table)

with engine.connect() as connection:
    results = connection.execute(alchemy_query).fetchall()
    print(results)

[(1, 'Alice', 'Johnson'), (2, 'Bob', 'Smith'), (3, 'Charlie', 'Brown'), (4, 'Diana', 'Prince'), (5, 'Evan', 'Taylor')]


In [10]:
# Sample data read with SQL query like format
from sqlalchemy import text

with engine.connect() as connection:
    rows = connection.execute(text("SELECT employee_id from employee"))
    for row in rows:
        print(row)

(1,)
(2,)
(3,)
(4,)
(5,)


After building the SQL database, we can utilize the NLSQLTableQueryEngine to create natural language queries, which are then converted into SQL queries. It's important to specify the tables we want the query engine to focus on. If we don't specify them, the engine will pull the entire schema context, which could exceed the context window of the LLM.

## Natural Language to SQL Query Engine (NLSQL)

In [11]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,  # The SQL database object containing the employee table
    tables=["employee"],  # The list of tables to query from (in this case, just "employee")
    llm=llm
)

# Define a query that asks how many employees have a salary greater than 70,000.
query_str = "How many employees have salary greater than 70000?"

# Use the query engine to process the query and get the response.
response = query_engine.query(query_str)

# Display the response returned by the query engine (e.g., the answer to the query).
display(response)

Response(response='There are 3 employees with a salary greater than $70,000.', source_nodes=[NodeWithScore(node=TextNode(id_='cc0d8ee8-8865-4d11-983e-206b3cbc7f5f', embedding=None, metadata={'sql_query': 'SELECT COUNT(*) AS employee_count FROM employee WHERE salary > 70000;', 'result': [(3,)], 'col_keys': ['employee_count']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='[(3,)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)], metadata={'cc0d8ee8-8865-4d11-983e-206b3cbc7f5f': {'sql_query': 'SELECT COUNT(*) AS employee_count FROM employee WHERE salary > 70000;', 'result': [(3,)], 'col_keys': ['employee_count']}, 'sql_query': 'SELECT COUNT(*) AS employee_count FROM employee WHERE salary > 70000;', 'result': [(3,)], 'col_key

In [12]:
# To get the response
print(response.response)

# To get the result of sql query
print(response.metadata["result"])

There are 3 employees with a salary greater than $70,000.
[(3,)]


If we're unsure in advance which table to use, and the full schema size exceeds the context window of the model, it's best to store the table schema in an index. This way, we can retrieve the appropriate schema during query execution.

To achieve this, we can use the SQLTableNodeMapping object, which takes a SQLDatabase and generates a Node object for each SQLTableSchema object. These nodes are then passed into the ObjectIndex constructor for indexing and retrieval.

## NLSQL with Table Schema stored in a Index

In [13]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine
from llama_index.core.objects import SQLTableNodeMapping,ObjectIndex,SQLTableSchema
from llama_index.core import VectorStoreIndex

# manually set context text for the table
context_str = (
    "This table gives information regarding the employees of the organization"
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="employee",context_str=context_str))
]  # add a SQLTableSchema if more tables are present

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [14]:
response = query_engine.query("Who are the employees have salary greater than 70000?")
display(response)

Response(response='The employees with a salary greater than $70,000 are Bob Smith with a salary of $120,000, Charlie Brown with a salary of $95,000, and Alice Johnson with a salary of $85,000.', source_nodes=[NodeWithScore(node=TextNode(id_='e6c02732-937f-456f-b986-1d79af664688', embedding=None, metadata={'sql_query': 'SELECT first_name, last_name, salary\nFROM employee\nWHERE salary > 70000\nORDER BY salary DESC;', 'result': [('Bob', 'Smith', 120000.0), ('Charlie', 'Brown', 95000.0), ('Alice', 'Johnson', 85000.0)], 'col_keys': ['first_name', 'last_name', 'salary']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="[('Bob', 'Smith', 120000.0), ('Charlie', 'Brown', 95000.0), ('Alice', 'Johnson', 85000.0)]", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\

In [15]:
response.response

'The employees with a salary greater than $70,000 are Bob Smith with a salary of $120,000, Charlie Brown with a salary of $95,000, and Alice Johnson with a salary of $85,000.'

In [16]:
response.metadata["result"]

[('Bob', 'Smith', 120000.0),
 ('Charlie', 'Brown', 95000.0),
 ('Alice', 'Johnson', 85000.0)]